# INDIGO fund geographic area for outcome payment


## INDIGO database API endpoint
Setup INDIGO database API endpoint and helper method for getting individual items from the API. This can be used with the project, fund, organisation and assessment_resource endpoints.

In [ ]:
!pip install pycountry

import requests
import pycountry

import numpy as np

import plotly.graph_objects as go


INDIGO_DATABASE_API = 'https://golab-indigo-data-store.herokuapp.com/app/api1'


def api_get_item(endpoint, public_id=None):
    """
    Get individual item details from the API

    E.g. 
    item = api_get_item('project', 'INDIGO-POJ-0158')
    """
    try:
        if public_id:
            response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}/{public_id}')
        else:
            response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}')
        item = response.json()
        return item
    except Exception as e:
        print(f'\nFailed to retrieve {endpoint} "{public_id}".\nError: {e}')
        return False


## Get project data
Call the INDIGO API 'project' endpoint and retrieve the data used for the plot.

In [ ]:
data = {}
endpoint = 'fund'

funds = api_get_item(endpoint=endpoint)['funds']

for fund in funds:
    if fund['public']:
        public_id = fund['id']
        data[public_id] = api_get_item(endpoint, public_id)['fund']['data']

# print(data)

In [ ]:
countries = {}

num_funds = len(data)

print('Number of funds:', num_funds)

countries = {}

for public_id, fund in data.items():
    areas = fund.get('geographic_areas_for_outcome_payment', [])

    if not areas:
        print('Missing areas:', public_id)

    for area in areas:
        alpha_2 = area.get('country', {}).get('value')

        if alpha_2:
            country = pycountry.countries.get(alpha_2=alpha_2)
            alpha_3 = country.alpha_3
            name = country.name
            # print(public_id, country, alpha_2)

            if alpha_3 in countries:
                countries[alpha_3]['count'] += 1 
            else:
                countries[alpha_3] = {
                    'name': name,
                    'count': 1,
                    'alpha_2': alpha_2,
                }

print(countries)

locations = []
names = []
counts = []

for alpha_3, value in countries.items():
    locations.append(alpha_3)
    
    name = value['name']
    count = value['count']
    alpha_2 = value['alpha_2']

    names.append(name)
    counts.append(count) 

In [ ]:
colorbar = dict(
    title='Outcome<br>Payments',
    title_font_size=15,
    title_side='right',
    thicknessmode='pixels', 
    thickness=20,
    ticks='outside',
    
)

fig = go.Figure(
    data=go.Choropleth(
        locations=locations,
        text=names,
        z=counts,
        locationmode='ISO-3',
        colorscale='teal',
        colorbar=colorbar,
        hovertemplate='%{text}: %{z}<extra></extra>',
    )
)

fig.update_layout(
    height=700,
    width=1200,
    title_text = 'INDIGO Funds - Geographic Area for Outcome Payment',
    title_font_size=24,
    title_x=0.5,
    margin={'t': 140, 'b': 0, 'l': 0, 'r': 0},
    hoverlabel={'bgcolor': 'white', 'font_size': 16},
)

fig.show()

## Important Notice and Disclaimer on INDIGO Data
<sub><sup>
INDIGO data are shared for research and policy analysis purposes. INDIGO data can be used to support a range of insights, for example, to understand the social outcomes that projects aim to improve, the network of organisations across projects, trends, scales, timelines and summary information. The collaborative system by which we collect, process, and share data is designed to advance data-sharing norms, harmonise data definitions and improve data use. These data are NOT shared for auditing, investment, or legal purposes. Please independently verify any data that you might use in decision making. We provide no guarantees or assurances as to the quality of these data. Data may be inaccurate, incomplete, inconsistent, and/or not current for various reasons: INDIGO is a collaborative and iterative initiative that mostly relies on projects all over the world volunteering to share their data. We have a system for processing information and try to attribute data to named sources, but we do not audit, cross-check, or verify all information provided to us. It takes time and resources to share data, which may not have been included in a project’s budget. Many of the projects are ongoing and timely updates may not be available. Different people may have different interpretations of data items and definitions. Even when data are high quality, interpretation or generalisation to different contexts may not be possible and/or requires additional information and/or expertise. Help us improve our data quality: email us at indigo@bsg.ox.ac.uk if you have data on new projects, changes or performance updates on current projects, clarifications or corrections on our data, and/or confidentiality or sensitivity notices. Please also give input via the INDIGO Data Definitions Improvement Tool and INDIGO Feedback Questionnaire.
</sup></sub>